In [ ]:
! wget https://raw.githubusercontent.com/vsubbian/WindowSHAP/main/windowshap.py -P utils/

In [ ]:
# conda activate uci_38
# conda install -c conda-forge aeon
# conda install shap lime
# pip install windowshap --no-deps
# pip install anchor-exp --no-deps
# conda install flask

In [1]:
%load_ext autoreload
%autoreload 2
# KERNEL = eb0c...

In [2]:
ls

auxiliary_mushrooms.py             mushroom-serialised/
duplicity-png.txt                  pkl_luiz.pkl
environment.yml                    __pycache__/
extracted_data/                    README.txt@
fast/                              results/
formatted_table_image.png          res_usage.sh*
intel/                             shap_values_X_val_cache.pkl
logs/                              shared/
models/                            slow/
mtm_ecg_results_v1_with_split.csv  utils/
MushroomDataset/                   XAI-ISI/


In [3]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # 0 - first gpu, 1 - second, "0,1" - both gpu, first used

import numpy as np

np.object = object
np.int = int
np.float = float
np.bool = bool
np.typeDict = np.sctypeDict

import gc
import multiprocessing
import numpy as np
import pandas as pd
import pickle
import requests
import shap
import tensorflow as tf
import tensorflow.keras as keras  # import keras
import warnings
import sys
import time
import random
import tempfile
import string
import traceback
import json
import alibi

from anchor import anchor_tabular  # pip install anchor-exp --no-deps
from alibi.explainers import AnchorTabular
from datetime import datetime
from aeon.datasets import load_classification
from lime import lime_tabular
from numpy import mean, std
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import backend as K
# from tensorflow.keras.layers import ConvLSTM1D  # only for h5 and does not work with new tf
from tensorflow.keras.utils import to_categorical
from tensorflow.python.framework import ops
from tensorflow.keras.initializers import glorot_uniform  # only for h5 
from tensorflow.python.util import deprecation
from typing import Any, Dict, List, Optional, Set, Tuple
from utils.windowshap import SlidingWindowSHAP
from multiprocessing import Queue, Lock
from concurrent.futures import ProcessPoolExecutor, TimeoutError as FuturesTimeoutError

print(tf.__version__)  # print(tf.keras.__version__)
print(alibi.explainers.__all__)

tempfile.tempdir = "fast/tmp"
os.makedirs(tempfile.tempdir, exist_ok=True)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
2025-03-18 11:06:20.436981: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-18 11:06:21.285289: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.1
['ALE', 'AnchorTabular', 'DistributedAnchorTabular', 'AnchorText', 'AnchorImage', 'CEM', 'Counterfactual', 'CounterfactualProto', 'CounterfactualRL', 'CounterfactualRLTabular', 'plot_ale', 'IntegratedGradients', 'KernelShap', 'TreeShap', 'GradientSimilarity']


In [4]:
tf.compat.v1.disable_v2_behavior()

warnings.filterwarnings('ignore')
ops.logging.set_verbosity(ops.logging.ERROR)
tf.get_logger().setLevel('FATAL')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
deprecation._PRINT_DEPRECATION_WARNINGS = False
os.environ['TF_ENABLE_DEPRECATION_WARNINGS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

gpus = tf.config.list_physical_devices("GPU")
print(gpus)
if gpus:
    gpu_to_limit = gpus[0]  # or 1, watch out for CUDA_VISIBLE_DEVICES
    print(gpu_to_limit)

Instructions for updating:
non-resource variables are not supported in the long term
[]


2025-03-18 11:06:23.909467: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-03-18 11:06:23.909495: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: ba97f2f42a1e
2025-03-18 11:06:23.909501: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: ba97f2f42a1e
2025-03-18 11:06:23.909695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 550.90.7
2025-03-18 11:06:23.909716: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 550.90.7
2025-03-18 11:06:23.909722: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 550.90.7


In [ ]:
if gpus:
    gpu_mem_mb = 24564  # tyle mamy
    tf.config.set_logical_device_configuration(
        gpu_to_limit, [tf.config.LogicalDeviceConfiguration(memory_limit=gpu_mem_mb // 10)]
    )

    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

data_path = "shared/UCI-Benchmark/"
batch_size = 64

In [21]:
# MODEL_MANAGER = "http://192.168.48.1:5005"
# MODEL_MANAGER = "http://172.16.216.132:5005"
MODEL_MANAGER = "http://172.16.216.132:6005"

# Real Data example

In [6]:
# convlstm model
def load_dataset_aeon(dsname):
    X, y, meta_data = load_classification(dsname)
    X = np.moveaxis(X, 1, 2)

    le = LabelEncoder()
    y = le.fit_transform(y)
    trainX, testX, trainy, testy = train_test_split(X, y)

    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    return trainX, trainy, testX, testy


# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))


# run an experiment
def train_model(trainX, trainy, testX, testy):
    return 1.0, keras.models.Sequential([])


def train_lstmconv_model(trainX, trainy, testX, testy):
    return 1.0, keras.models.Sequential([])


In [7]:
# @tf.function
def evaluate_model(model, testX, testy, batch_size, steps):
    # f = io.StringIO()
    # with redirect_stdout(f), redirect_stderr(f):
    # sys.stderr = open(os.devnull, 'w')
    if steps is not None:
        result = model.evaluate(testX, testy, batch_size=batch_size, verbose=0, steps=steps)
    else:
        result = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    # sys.stderr = sys.__stderr__
    return result

## save & load bundle

In [8]:
def save_bundle(model, trainX, trainy, testX, testy, svtr, svts, dsname,
                dir='./results', explain_prefix="sv", only_explain=False):
    if not os.path.isdir(f'{dir}/{dsname}'):
        os.makedirs(f'{dir}/{dsname}')

    if not only_explain:
        # model.save(f'{dir}/{dsname}/model.h5', save_format='h5')
        model.save(f'{dir}/{dsname}/model_tf', save_format='tf')

        pickle.dump(trainX, open(f'{dir}/{dsname}/trainX.pickle', 'wb'))
        pickle.dump(trainy, open(f'{dir}/{dsname}/trainy.pickle', 'wb'))
        pickle.dump(testX, open(f'{dir}/{dsname}/testX.pickle', 'wb'))
        pickle.dump(testy, open(f'{dir}/{dsname}/testy.pickle', 'wb'))

    if svtr is not None:
        pickle.dump(svtr, open(f'{dir}/{dsname}/{explain_prefix}tr.pickle', 'wb'))
    if svts is not None:
        pickle.dump(svts, open(f'{dir}/{dsname}/{explain_prefix}ts.pickle', 'wb'))


def load_bundle(dsname, dir='./results', explain_prefix="sv", only_explain=False, load_model=True):
    model, model_graph, trainX, trainy, testX, testy = None, None, None, None, None, None,
    if not only_explain:
        tf_model_path = f'{dir}/{dsname}/model_tf/1'
        h5_model_path = f'{dir}/{dsname}/model.h5'

        # TODO model_graph does not help in multiprocessing, but when once created, must be used, so I don't use it!
        # model_graph = tf.Graph()
        # with model_graph.as_default():
        if load_model:
            if os.path.exists(tf_model_path):
                print("Loading model in tf format...")
                model = tf.keras.models.load_model(tf_model_path)
            elif os.path.exists(h5_model_path):
                print("Loading model in h5 format...")
                model = tf.keras.models.load_model(
                    h5_model_path, custom_objects={'GlorotUniform': glorot_uniform, 'ConvLSTM1D': ConvLSTM1D})
                # Save model in tf format if loaded from h5
                if not os.path.exists(tf_model_path):
                    os.makedirs(tf_model_path, exist_ok=True)
                    model.save(tf_model_path, save_format='tf')
                    print(f"Model {dsname} saved in TensorFlow format at {tf_model_path}")
            else:
                raise FileNotFoundError(
                    f'Nie znaleziono modelu w formatach SavedModel ani H5 dla {dsname} w katalogu {dir}.')
            # print(model.summary())

        trainX = pickle.load(open(f'{dir}/{dsname}/trainX.pickle', 'rb'))
        trainy = pickle.load(open(f'{dir}/{dsname}/trainy.pickle', 'rb'))
        testX = pickle.load(open(f'{dir}/{dsname}/testX.pickle', 'rb'))
        testy = pickle.load(open(f'{dir}/{dsname}/testy.pickle', 'rb'))

    svtr = pickle.load(open(f'{dir}/{dsname}/{explain_prefix}tr.pickle', 'rb'))
    svts = pickle.load(open(f'{dir}/{dsname}/{explain_prefix}ts.pickle', 'rb'))

    return model, model_graph, trainX, trainy, testX, testy, svtr, svts

## Univariate

In [9]:
#list directoies in  folder
datasets_uni = [f.name for f in os.scandir(data_path + 'ds/univariate') if f.is_dir() and not f.name.startswith('.')]
print(datasets_uni)

['DodgerLoopGame', 'ProximalPhalanxTW', 'DodgerLoopDay', 'ECGFiveDays', 'UMD', 'Plane', 'ECG200', 'TwoPatterns', 'GunPointAgeSpan', 'DiatomSizeReduction', 'UWaveGestureLibraryAll', 'MedicalImages', 'Meat', 'Trace', 'MiddlePhalanxOutlineAgeGroup', 'Chinatown', 'DistalPhalanxOutlineAgeGroup', 'WormsTwoClass', 'DistalPhalanxOutlineCorrect', 'Strawberry', 'OliveOil', 'UWaveGestureLibraryX', 'SmoothSubspace', 'Fungi', 'ElectricDevices', 'SwedishLeaf', 'CricketX', 'ECG5000', 'PhalangesOutlinesCorrect', 'FaceFour', 'SyntheticControl', 'FordA', 'PowerCons', 'BeetleFly', 'GunPointMaleVersusFemale', 'Yoga', 'Herring', 'Crop', 'RefrigerationDevices', 'Worms', 'OSULeaf', 'ItalyPowerDemand', 'GunPoint', 'CBF', 'Symbols', 'ToeSegmentation2', 'TwoLeadECG', 'SmallKitchenAppliances', 'ShapesAll', 'ScreenType', 'Computers', 'CricketY', 'SonyAIBORobotSurface2', 'Wafer', 'MiddlePhalanxOutlineCorrect', 'Lightning2', 'UWaveGestureLibraryY', 'Wine', 'ProximalPhalanxOutlineCorrect', 'ShapeletSim', 'CricketZ',

In [10]:
for p in datasets_uni:
    print(f'Loading... {p}')
    ##trainsvo is shap calculated for train, testsv is shap calculated for test (NOTE: these are absolute values of shap)
    model, _, trainXo, trainyo, testX, testy, trainsvo, testsv = load_bundle(p,
                                                                             dir=data_path + 'ds/univariate')
    # with HiddenPrints():
    _, score = evaluate_model(model, testX, testy, batch_size=batch_size, steps=None)
    score = score * 100.0
    print('>Accuracy: %.3f' % (score))

    K.clear_session()
    del model, trainXo, trainyo, testX, testy, trainsvo, testsv
    gc.collect()

    break


Loading... DodgerLoopGame
Loading model in tf format...


2025-03-18 11:07:26.934475: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-03-18 11:07:26.982976: W tensorflow/c/c_api.cc:304] Operation '{name:'AssignVariableOp_13' id:281 op device:{requested: '/device:CPU:0', assigned: ''} def:{{{node AssignVariableOp_13}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false, _device="/device:CPU:0"](dense_1/kernel/v, Identity_13)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2025-03-18 11:07:27.130845: W tensorflow/c/c_api.cc:304] Operation '{name:'loss/mul' id:986 op device:{requested: '', assigned: ''} def:{{{node loss/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul/x, loss/dense_1_loss/value)}}' was changed by setting attribute after it was run by a sessi

>Accuracy: 87.500


## Multivariate

In [11]:
#list directoies in  folder
datasets_multi = [f.name for f in os.scandir(data_path + '/ds/multivariate') if
                  f.is_dir() and not f.name.startswith('.')]
print(datasets_multi)

['ArticularyWordRecognition', 'Handwriting', 'BasicMotions', 'HandMovementDirection', 'Heartbeat', 'FaceDetection', 'SelfRegulationSCP2', 'NATOPS', 'ERing', 'EthanolConcentration', 'FingerMovements', 'Epilepsy', 'Libras', 'AtrialFibrillation', 'PenDigits', 'Cricket', 'LSST', 'UWaveGestureLibrary', 'RacketSports', 'SelfRegulationSCP1']


In [12]:
for p in datasets_multi:
    print(f'Loading... {p}')
    model, _, trainXo, trainyo, testX, testy, trainsvo, testsv = load_bundle(p,
                                                                             dir=data_path + 'ds/multivariate')
    # steps = len(testX) // batch_size
    # if len(testX) % batch_size != 0:
    #     steps += 1
    _, score = evaluate_model(model, testX, testy, batch_size, None)
    score = score * 100.0
    print('>Accuracy: %.3f' % (score))

    # K.clear_session()
    # del model, trainXo, trainyo, testX, testy, trainsvo, testsv
    # gc.collect()

    break

Loading... ArticularyWordRecognition
Loading model in tf format...


2025-03-18 11:07:35.411988: W tensorflow/c/c_api.cc:304] Operation '{name:'AssignVariableOp_20' id:301 op device:{requested: '/device:CPU:0', assigned: ''} def:{{{node AssignVariableOp_20}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false, _device="/device:CPU:0"](conv_lstm1d/kernel/m, Identity_20)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2025-03-18 11:07:35.551316: W tensorflow/c/c_api.cc:304] Operation '{name:'loss/mul' id:992 op device:{requested: '', assigned: ''} def:{{{node loss/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul/x, loss/dense_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or c

>Accuracy: 96.528


In [13]:
dir(model)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 3, 48, 9)          0         
                                                                 
 conv_lstm1d (ConvLSTM1D)    (None, 3, 48, 64)         168448    
                                                                 
 conv_lstm1d_1 (ConvLSTM1D)  (None, 3, 48, 32)         110720    
                                                                 
 dropout (Dropout)           (None, 3, 48, 32)         0         
                                                                 
 embedding (Flatten)         (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 100)               460900    
                                                                 
 dense_1 (Dense)             (None, 25)                2

In [14]:
print(trainXo.shape)
print(trainsvo.shape)

(431, 144, 9)
(431, 144, 9)


# Sample

In [14]:
# nvidia-smi
# htop

# ps aux | grep model_server_gpu
#                                   pkill -f model_server_gpu

# ports=$(ps aux | grep model_server | grep -- "--port" | awk -F '--port ' '{print $2}' | awk '{print $1}' | sort -n); echo "Liczba portów: $(echo "$ports" | wc -l), Najniższy port: $(echo "$ports" | head -n 1), Najwyższy port: $(echo "$ports" | tail -n 1)"

# ps aux | grep model_server | grep -- "--port" | awk -F '--port ' '{print $2}' | awk '{print $1}' | sort -n | tr '\n' ' '

In [12]:
# tf.keras.backend.clear_session()
# gc.collect()

# directory, timeseries = "univariate", "DodgerLoopGame"
# directory, timeseries = "univariate", "Wafer"
# directory, timeseries = "univariate", "Crop"
directory, timeseries = "multivariate", "SelfRegulationSCP1"
# directory, timeseries = "multivariate", "ArticularyWordRecognition"
# directory, timeseries = "multivariate", "Heartbeat"
_, _, trainX_new, trainy_new, testX_new, testy_new, _, _ = load_bundle(
    timeseries, dir=os.path.join(data_path, 'ds', directory), load_model=False)

n_timesteps_new, n_features_new = trainX_new.shape[1], trainX_new.shape[2]
print(n_timesteps_new, n_features_new)

trainX_reshaped_new = trainX_new.reshape(-1, n_timesteps_new * n_features_new)
index_new = 108
instance_new = trainX_reshaped_new[index_new].reshape(-1, n_timesteps_new, n_features_new)

print(instance_new.shape)
print(instance_new[:, :10, :])
type(instance_new)

896 6
(1, 896, 6)
[[[-0.15586902 -0.25176516  0.02883643 -0.04777336 -0.0622212
   -0.2121953 ]
  [-0.2401376  -0.19744996  0.06404643 -0.06184454 -0.06966394
   -0.20806369]
  [-0.29742364 -0.09628271  0.0847582  -0.08129587 -0.09323261
   -0.12708425]
  [-0.36218176 -0.0626985   0.06653184 -0.06184454 -0.1473992
   -0.04817061]
  [-0.40908989  0.00737225  0.06528914 -0.03080518 -0.19908488
    0.03446147]
  [-0.38708874 -0.0083833   0.00688196 -0.04363478 -0.21727824
    0.12370412]
  [-0.25840272  0.02271319 -0.05401064 -0.06846627 -0.2185187
    0.15469115]
  [-0.20402249  0.05795588 -0.06809464 -0.07591571 -0.21438384
    0.15179903]
  [-0.10273414  0.07868687 -0.0875637  -0.09950562 -0.1333407
    0.21914417]
  [-0.06910973  0.0604436  -0.06809464 -0.15372104 -0.05436498
    0.22575474]]]


numpy.ndarray

# Explainers

In [15]:
def prepare_data(file_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path, header=None, sep='|')
    df = df[0].str.split(',', expand=True).dropna(axis=1)
    df.columns = df.iloc[0]
    return df[1:]


def load_background_data(trainX: np.ndarray, bg_size: int) -> np.ndarray:
    indexes = np.arange(len(trainX))
    np.random.shuffle(indexes)
    maxid = min(bg_size, len(trainX))
    return trainX[indexes[:maxid]]


def compute_window_shap(model: Any, trainX: np.ndarray, testX: np.ndarray, window_len: int, stride: int,
                        bg_data: np.ndarray, output_shape: int) -> Tuple[np.ndarray, np.ndarray]:
    sv_ts = np.zeros((len(testX), testX.shape[1], testX.shape[2]))
    sv_tr = np.zeros((len(trainX), trainX.shape[1], trainX.shape[2]))

    for i in range(len(testX)):
        gtw = SlidingWindowSHAP(model, stride, window_len, bg_data, testX[i:i + 1], model_type='lstm')
        sv_ts[i, :, :] = gtw.shap_values(num_output=output_shape)

    for i in range(len(trainX)):
        gtw = SlidingWindowSHAP(model, stride, window_len, bg_data, trainX[i:i + 1], model_type='lstm')
        sv_tr[i, :, :] = gtw.shap_values(num_output=output_shape)

    return sv_tr, sv_ts


def compute_deep_shap(model: Any, trainX: np.ndarray, testX: np.ndarray, bg_data: np.ndarray, absshap: bool) -> Tuple[
    np.ndarray, np.ndarray]:
    explainer = shap.DeepExplainer(model, bg_data)
    shap_values_ts = explainer.shap_values(testX, check_additivity=False)
    shap_values_tr = explainer.shap_values(trainX, check_additivity=False)

    if absshap:
        return abs(np.array(shap_values_tr)).mean(axis=0), abs(np.array(shap_values_ts)).mean(axis=0)

    indexer_ts = np.argmax(model.predict(testX), axis=1)
    sv_ts = np.concatenate([shap_values_ts[indexer_ts[i]][i, :] for i in range(len(testX))])

    indexer_tr = np.argmax(model.predict(trainX), axis=1)
    sv_tr = np.concatenate([shap_values_tr[indexer_tr[i]][i, :] for i in range(len(trainX))])

    return sv_tr, sv_ts


def compute_deep_lime(model: Any, trainX: np.ndarray, testX: np.ndarray) -> Tuple[
    np.ndarray, np.ndarray]:
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]

    trainX_reshaped = trainX.reshape(-1, n_timesteps * n_features)
    testX_reshaped = testX.reshape(-1, n_timesteps * n_features)

    nan_count_train_X = np.isnan(trainX_reshaped).sum()
    nan_count_test_X = np.isnan(testX_reshaped).sum()
    total_train_elements = trainX_reshaped.size
    total_test_elements = testX_reshaped.size

    if nan_count_train_X > 0 or nan_count_test_X > 0:
        nan_percentage_train_X = (nan_count_train_X / total_train_elements) * 100
        nan_percentage_test_X = (nan_count_test_X / total_test_elements) * 100
        print(f"Warning: Found NaN values. Train NaNs: {nan_count_train_X} ({nan_percentage_train_X:.2f}%), "
              f"Test NaNs: {nan_count_test_X} ({nan_percentage_test_X:.2f}%)")

        most_frequent_train_X = np.nanmedian(trainX_reshaped)

        # Calculate the most frequent value across both samples
        combined_data_X = np.concatenate((trainX_reshaped[~np.isnan(trainX_reshaped)],
                                          testX_reshaped[~np.isnan(testX_reshaped)]))
        most_frequent_value_X = np.bincount(combined_data_X.astype(int)).argmax()

        print(
            f"Replacing NaN values with the most frequent value in train: {most_frequent_train_X}, while most frequent overall is  {most_frequent_value_X}")

        trainX_reshaped = np.where(np.isnan(trainX_reshaped), most_frequent_train_X, trainX_reshaped)
        testX_reshaped = np.where(np.isnan(testX_reshaped), most_frequent_train_X, testX_reshaped)

    train_predictions = model.predict(trainX.reshape(-1, n_timesteps, n_features))
    test_predictions = model.predict(testX.reshape(-1, n_timesteps, n_features))

    print("# of non-Nan predictions for train", np.count_nonzero(~np.isnan(train_predictions)),
          "of", train_predictions.size)
    print("# of non-Nan predictions for test", np.count_nonzero(~np.isnan(test_predictions)),
          "of", test_predictions.size)

    nan_threshold_y = 1.0  # percent
    nan_count_train_y = np.isnan(train_predictions).sum()
    nan_count_test_y = np.isnan(test_predictions).sum()

    nan_percentage_train_y = (nan_count_train_y / train_predictions.size) * 100
    nan_percentage_test_y = (nan_count_test_y / test_predictions.size) * 100

    if nan_percentage_train_y > nan_threshold_y or nan_percentage_test_y > nan_threshold_y:
        print(f"Error: NaN percentage in predictions exceeded threshold. "
              f"Train NaNs: {nan_count_train_y} ({nan_percentage_train_y:.2f}%), "
              f"Test NaNs: {nan_count_test_y} ({nan_percentage_test_y:.2f}%).")
        print("Stopping LIME calculations and returning empty results.")

        return np.empty((len(trainX), n_timesteps, n_features)), np.empty(
            (len(testX), n_timesteps, n_features))  # return np.array([]), np.array([])
    elif nan_percentage_train_y > 0.0 or nan_percentage_test_y > 0.0:
        print(f"Warning: There are NaN's in predictions. "
              f"Train NaNs: {nan_count_train_y} ({nan_percentage_train_y:.2f}%), "
              f"Test NaNs: {nan_count_test_y} ({nan_percentage_test_y:.2f}%).")

    most_frequent_value_y = np.nanmedian(
        train_predictions)  # np.bincount(train_predictions[~np.isnan(train_predictions)].astype(int)).argmax()

    def safe_predict_fn(x):
        prediction = model.predict(x.reshape(-1, n_timesteps, n_features))

        # Check for NaNs in the prediction
        if np.isnan(prediction).sum() > 0:
            nan_count_prediction = np.isnan(prediction).sum()
            nan_percentage_prediction = (nan_count_prediction / prediction.size) * 100
            # Replace NaNs in the prediction with a default value  
            prediction = np.nan_to_num(prediction, nan=most_frequent_value_y)
            print(f"Warning: Prediction contains NaN values "
                  f"({nan_count_prediction} NaNs, {nan_percentage_prediction:.2f}%). "
                  f"Replacing NaN values in prediction with {most_frequent_value_y}.")

        return prediction

    feature_names = [f'feature_{i}' for i in range(n_timesteps * n_features)]
    class_names = [f'class_{i}' for i in range(model.output_shape[-1])]

    explainer = lime_tabular.LimeTabularExplainer(
        training_data=trainX_reshaped,
        feature_names=feature_names,
        class_names=class_names,
        mode="classification",
        discretize_continuous=True
    )

    sv_ts = []
    sv_tr = []

    for i in range(len(testX)):
        # for i in tqdm(range(len(testX)), desc="Processing test data"):
        # print(f"[{i}] testX: start")
        explanation = explainer.explain_instance(
            data_row=testX_reshaped[i],
            predict_fn=safe_predict_fn,
            num_features=n_timesteps * n_features
        )
        weights = np.array([feature[1] for feature in explanation.as_list()])
        shaped_weights = weights.reshape((n_timesteps, n_features))
        sv_ts.append(shaped_weights)

    for i in range(len(trainX)):
        # for i in tqdm(range(len(trainX)), desc="Processing train data"):
        explanation = explainer.explain_instance(
            data_row=trainX_reshaped[i],
            predict_fn=safe_predict_fn,  # lambda x: model.predict(x.reshape(-1, n_timesteps, n_features))
            num_features=n_timesteps * n_features
        )
        weights = np.array([feature[1] for feature in explanation.as_list()])
        shaped_weights = weights.reshape((n_timesteps, n_features))
        sv_tr.append(shaped_weights)

    return np.array(sv_tr), np.array(sv_ts)


def process_dataset(
        port_queue: Optional[Queue], port_lock: Optional[Lock], model_name: str, directory: str, model_path: str,
        model: Any, available_ports: Optional[List[int]], trainX: np.ndarray, trainy: np.ndarray, testX: np.ndarray,
        testy: np.ndarray, bg_size: int, window_len: int, stride: int, explanation_method: str, absshap: bool,
        shap_enabled: bool, lime_enabled: bool, anchor_enabled: bool, timeout_duration: int, alt_timeout_duration: int,
        parallel: bool = True) -> \
        Tuple[
            Optional[np.ndarray], Optional[np.ndarray], Optional[np.ndarray], Optional[np.ndarray], Optional[
                List[Dict]],
            Optional[List[Dict]]]:
    print(f"Processing... {p} with trainX {trainX.shape} and testX {testX.shape} ")

    bg_data = load_background_data(trainX, bg_size)  # beginning of dataset

    sv_tr, sv_ts = None, None
    lime_tr, lime_ts = None, None
    anchor_tr, anchor_ts = None, None

    if shap_enabled:
        if explanation_method == 'window':
            print("SHAP: window")
            sv_tr, sv_ts = compute_window_shap(model, trainX, testX, window_len, stride, bg_data, trainy.shape[1])
        else:
            print("SHAP: deep")
            sv_tr, sv_ts = compute_deep_shap(model, trainX, testX, bg_data, absshap)

    if lime_enabled:
        print("LIME: deep")
        lime_tr, lime_ts = compute_deep_lime(model, trainX, testX)

    if anchor_enabled:
        print("ANCHOR: ...")
        anchor_tr, anchor_ts = compute_anchor(port_queue, port_lock, model_name, directory, model_path, model,
                                              available_ports, trainX, testX, bg_data, parallel, timeout_duration,
                                              alt_timeout_duration, max_attempts=5)

    return sv_tr, sv_ts, lime_tr, lime_ts, anchor_tr, anchor_ts

## ANCHOR

### Parallel with TensorFlow Serving

In [12]:
#    source ~/miniconda3/bin/activate
#    conda create -n tensorflow-serving-env python=3.9 -y
#    conda activate tensorflow-serving-env
#    conda install tensorflow tensorflow-gpu psutil flask requests -y
#    python -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))"


#### ModelServingClient

In [16]:
class ModelServingClient:
    def __init__(self, base_url: str, base_port: int, predict_url: str = "http://172.16.216.132", timeout: int = 30):
        self.base_url = base_url
        self.base_port = base_port
        self.predict_url = predict_url
        self.timeout = timeout
        self.used_ports: Set = set()  # Track used ports
        self.checked_ports: Set = set()
        self.used_ports_info = []
        self.method: Optional[str] = None
        self.grow_gpu_mem: Optional[str] = None
        self.model: Optional[str] = None

    def update_used_ports(self, model_name: Optional[str] = None):
        """Fetch active serving containers and update self.used_ports."""
        url = f"{self.base_url}/active_serving_containers"
        try:
            response = requests.get(url, timeout=self.timeout)
            response.raise_for_status()  # Raise an error for bad status codes
            containers = response.json()

            # Update self.used_ports with model details
            self.used_ports.clear()
            self.used_ports_info = []
            for container in containers:
                if model_name is None or model_name == container['model_name']:
                    self.used_ports.add(container['port'])

                    self.used_ports_info.append({
                        "model_name": container["model_name"],
                        "gpu": container["gpu"],
                        "port": container["port"],
                        "container_id": container["container_id"],
                        "pid": container["pid"]},
                    )
                    if self.model is not None and self.model != container["model_name"]:
                        print(f"Model {self.model} mismatch with container ({container['model_name']})!")

            print("Updated used ports:", self.used_ports_info)

        except requests.RequestException as e:
            print(f"Failed to fetch active serving containers with model {model_name}: {e}")

    def find_available_port(self, actual_port: Optional[int] = None) -> int:
        """Finds the first available port starting from base_port."""
        port = actual_port or self.base_port
        while True:
            if port not in self.checked_ports and port not in self.used_ports:
                response = requests.get(f"{self.base_url}/check_ports", params={"port": port}, timeout=self.timeout)
                if response.status_code == 200:
                    self.used_ports.add(port)
                    return port
                self.checked_ports.add(port)
            port += 2

    def get_gpu_with_most_memory(self) -> int:
        """Returns the GPU index with the most available memory."""
        response = requests.get(f"{self.base_url}/gpu_memory", timeout=self.timeout)
        memory_info = response.json()
        print("memory_info:", memory_info)
        return max(memory_info, key=lambda gpu: int(memory_info[gpu].replace(" MiB", "")))

    def has_sufficient_gpu_memory(self, gpu: str, required_memory_gb: int) -> bool:
        """
        Checks if the specified GPU has at least the required amount of free memory.
    
        Parameters:
        - gpu (str): The GPU ID to check memory for.
        - required_memory_gb (int): Minimum required memory in GB.
    
        Returns:
        - bool: True if the GPU has sufficient memory, False otherwise.
        """
        try:
            response = requests.get(f"{self.base_url}/gpu_memory", timeout=self.timeout)
            response.raise_for_status()
            memory_info = response.json()

            # Sprawdź, czy wskazany GPU jest w odpowiedzi
            if gpu not in memory_info:
                print(f"GPU {gpu} not found in memory info: {memory_info}")
                return False

            free_memory_mib = int(memory_info[gpu].replace(" MiB", ""))

            free_memory_gb = free_memory_mib / 1024
            return free_memory_gb >= required_memory_gb

        except requests.RequestException as e:
            print(f"Error fetching GPU memory info: {str(e)}")
            return False
        except ValueError as e:
            print(f"Error parsing GPU memory info: {str(e)}")
            return False

    def load_models(self, model_directory: str, model_name: str, count: int, data: np.ndarray, method: str = "docker",
                    grow_gpu_mem: str = "true", gpu: Optional[str] = None, num_samples: int = 1_000,
                    req_free_memory_gb=1) -> Tuple[
        List[Dict[str, Any]], List[int]]:
        """Loads specified number of models with selected resources and returns details."""
        self.method = method
        self.grow_gpu_mem = grow_gpu_mem
        self.model = None

        loaded_models_info = []
        model_path = os.path.join("/opt/jupyterhub/fast/shared_dir/UCI-Benchmark", 'ds', model_directory, model_name,
                                  "model_tf")

        for _ in range(count):
            port = self.find_available_port()
            final_gpu = gpu or str(self.get_gpu_with_most_memory()) or "all"

            if final_gpu != "all" and not self.has_sufficient_gpu_memory(final_gpu,
                                                                         required_memory_gb=req_free_memory_gb):
                print(f"Insufficient GPU memory on {final_gpu}. Skipping model allocation.")
                continue

            model_info = self.load_model(model_name, model_path, port, final_gpu, method, grow_gpu_mem)
            if model_info["status"] == "loaded":
                time.sleep(3)
                self.wait_for_successful_prediction(model_info["port"], final_gpu, method, model_name, data,
                                                    num_samples)
                self.used_ports.add(port)
                self.model = model_name

            loaded_models_info.append(model_info)
            time.sleep(1)

        return loaded_models_info, list(self.used_ports)

    def load_model(self, model_name: str, model_path: str, port: int, gpu: str, method: str, grow_gpu_mem: str) -> Dict[
        str, Any]:
        """Helper function to send model loading request and return load status."""
        if self.model is None:
            self.model = model_name
        else:
            if self.model != model_name:
                raise Exception(f"Requested model {model_name} does not match defined self.model= {self.model}.")

        start_time = time.time()
        load_url = f"{self.base_url}/load_model"
        data = {
            "model_name": model_name,
            "model_path": model_path,
            "port": port,
            "gpu": gpu,
            "method": method,
            "grow_gpu_mem": grow_gpu_mem,
        }

        response = requests.post(load_url, json=data, timeout=self.timeout)
        try:
            response_json = response.json()
        except ValueError:
            response_json = {"error": response.text or "Response is not in JSON format"}

        if response.status_code == 200:
            return {
                "model_name": model_name,
                "port": port,
                "gpu": gpu,
                "load_time": time.time() - start_time,
                "status": "loaded"
            }
        else:
            return {
                "model_name": model_name,
                "port": port,
                "status": "failed",
                "error": response_json.get("error", "Unknown error")
            }

    def wait_for_successful_prediction(self, port: int, gpu: str, method: str, model_name: str, data: np.ndarray,
                                       num_samples) -> None:
        """Wait until a successful prediction is returned by the model."""
        check_url = f"{self.predict_url}:{port}/v1/models/{model_name}"
        predict_url = f"{self.predict_url}:{port}/v1/models/{model_name}:predict"
        success = False
        # Powiel dane, aby zwiększyć zużycie pamięci
        random_samples = np.random.normal(0, 1, (num_samples, *data.shape[1:]))
        test_data = np.concatenate([data, random_samples], axis=0)
        print("method:", method, ", port:", port, ", gpu:", gpu, ", model_name:", model_name, ", test_data:",
              test_data.shape)
        while not success:
            try:
                response = requests.get(check_url, timeout=self.timeout)
                if response.status_code == 200:
                    model_info = response.json()
                    if model_info.get("model_version_status", [{}])[0].get("state") == "AVAILABLE":
                        response = requests.post(predict_url, data=json.dumps({"instances": test_data.tolist()}),
                                                 timeout=self.timeout)
                        if response.status_code == 200:
                            success = True
                        if response.status_code == 400 and "Input to reshape is a tensor with" in response.json().get(
                                "error", ""):
                            raise Exception(response.json().get("error", ""))
                if not success:
                    print(response, response.json())
                    time.sleep(1)
            except (requests.RequestException, requests.ConnectionError) as e:
                print(f"Error while waiting for prediction: {e}")
                time.sleep(5)

    def reload_models(self, model_directory: str, model_name: str, method: str = "docker",
                      ports: Optional[List[int]] = None) -> List[int]:
        """Reloads models on all used ports and returns list of successful ports."""
        reloaded_ports = set()
        self.model = None
        model_path = os.path.join("/opt/jupyterhub/fast/shared_dir/UCI-Benchmark", 'ds', model_directory, model_name,
                                  "model_tf")

        if ports is None:
            ports = list(self.used_ports)

        for port in ports:
            reload_url = f"{self.base_url}/load_model"
            data = {
                "model_name": model_name,
                "model_path": model_path,
                "port": port,
                "method": method
            }
            try:
                response = requests.post(reload_url, json=data, timeout=self.timeout)

                if response.status_code == 200:
                    self.model = model_name
                    self.used_ports.add(port)
                    reloaded_ports.add(port)
                else:
                    reloaded_ports.remove(port)

            except requests.Timeout:
                print(f"Timeout: Port {port} may not be responding.")

        return list(reloaded_ports)

    def unload_models(self, ports: Optional[List[int]] = None) -> List[int]:
        """Unloads models on all used ports and returns list of successful ports."""
        if ports is None:
            ports = list(self.used_ports)

        unloaded_ports = set()
        for port in ports:
            unload_url = f"{self.base_url}/unload_model"
            data = {"port": port}
            response = requests.post(unload_url, json=data, timeout=self.timeout)
            if response.status_code == 200:
                unloaded_ports.add(port)
                self.model = None
            self.used_ports.discard(port)

        return list(unloaded_ports)

    def terminate_all_containers(self, model_name: Optional[str] = None) -> dict:
        """Send a request to terminate all running TensorFlow Serving containers."""
        url = f"{self.base_url}/terminate_all_containers"
        payload = {"model_name": model_name} if model_name else {}
        self.model = None
        try:
            response = requests.post(url, json=payload, timeout=self.timeout)
            response.raise_for_status()  # Raise an error for bad status codes
            return response.json()  # Return JSON response with terminated containers
        except requests.RequestException as e:
            print(f"Failed to terminate all containers: {e}")
            return {"error": str(e)}

    def predict(self, model_name: str, data: np.ndarray, index: int) -> Tuple[int, Dict[str, Any]]:
        """Sends a prediction request to the model server using the port determined by index modulo rotation."""
        if len(self.used_ports):
            # port = random.choice(list(self.used_ports)) # Choose a random port from available ports
            port = list(self.used_ports)[index % len(self.used_ports)]  # Choose constant
            data_dict = {"instances": data.tolist()}
            prediction_url = f"{self.predict_url}:{port}/v1/models/{model_name}:predict"
            try:
                response = requests.post(prediction_url, data=json.dumps(data_dict), timeout=self.timeout)
                if response.status_code == 200:
                    return port, response.json()
                else:
                    return port, {
                        "error": f"Failed to get prediction on port {port}. Status code: {response.status_code}"}

            except requests.Timeout:
                return port, {"error": f"Prediction request timed out on port {port}"}

            except requests.ConnectionError as e:
                # Check for specific "Connection reset by peer" error and terminate the container
                # if isinstance(e.args[0], requests.exceptions.ConnectionError):
                # print(f"Connection reset by peer on port {port}. Terminating container.")
                # self.terminate_container(port)  # Terminate the container on this port
                return port, {"error": f"Connection error on port {port}: {str(e)}"}

            except Exception as e:
                return port, {"error": f"Exception on port {port}: {str(e)}"}

        return -1, {"error": f"No port available"}

    def terminate_container(self, port: int) -> None:
        """Terminate a single container identified by the port."""
        url = f"{self.base_url}/terminate_container"
        try:
            response = requests.post(url, json={"port": port}, timeout=self.timeout)
            if response.status_code == 200:
                print(f"Container on port {port} terminated successfully.")
            else:
                print(f"Failed to terminate container on port {port}. Status code: {response.status_code}")
        except requests.RequestException as e:
            print(f"Failed to terminate container on port {port}: {e}")
        self.used_ports.remove(port)  # Remove port from used ports list


In [14]:
client = ModelServingClient(MODEL_MANAGER, base_port=8015)

In [15]:
client.get_gpu_with_most_memory()

memory_info: {'0': '11209 MiB', '1': '11208 MiB'}


'0'

In [290]:
client.terminate_all_containers()
client.update_used_ports()

{'message': 'Forcefully terminated 6 containers',
 'terminated_containers': [{'container_id': '9cf068d1e536',
   'gpu': '0',
   'model_name': 'Wafer',
   'pid': 1848066,
   'port': 8015},
  {'container_id': '9cf068d1e536',
   'gpu': '0',
   'model_name': 'Wafer',
   'pid': 1848066,
   'port': 8015},
  {'container_id': '20f9360fc68a',
   'gpu': '0',
   'model_name': 'Wafer',
   'pid': 1844552,
   'port': 8019},
  {'container_id': '20f9360fc68a',
   'gpu': '0',
   'model_name': 'Wafer',
   'pid': 1844552,
   'port': 8019},
  {'container_id': '731b3625375d',
   'gpu': '1',
   'model_name': 'SelfRegulationSCP1',
   'pid': 1814693,
   'port': 8017},
  {'container_id': '731b3625375d',
   'gpu': '1',
   'model_name': 'SelfRegulationSCP1',
   'pid': 1814693,
   'port': 8017}]}

In [292]:
client.update_used_ports()
model_info, tf_model_server_ports = client.load_models(
    directory, timeseries, data=instance_new, count=2, grow_gpu_mem="true", method="docker",
    gpu=None, num_samples=2_000)
print("Loaded models:", model_info)

Updated used ports: []
method: docker , port: 8015 , gpu: 1 , model_name: SelfRegulationSCP1 , test_data: (2001, 896, 6)
Error while waiting for prediction: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
method: docker , port: 8017 , gpu: 1 , model_name: SelfRegulationSCP1 , test_data: (2001, 896, 6)
Error while waiting for prediction: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Loaded models: [{'model_name': 'SelfRegulationSCP1', 'port': 8015, 'gpu': '1', 'load_time': 7.633980989456177, 'status': 'loaded'}, {'model_name': 'SelfRegulationSCP1', 'port': 8017, 'gpu': '1', 'load_time': 5.750410318374634, 'status': 'loaded'}]


In [294]:
# tf_model_server_ports = [mi['port'] for mi in model_info]
tf_model_server_ports

[8017, 8015]

In [286]:
tf_model_server_ports = client.reload_models("univariate", "Wafer")
print("Ports:", tf_model_server_ports)

Ports: [8019, 8015]


In [287]:
client.update_used_ports()
client.used_ports

Updated used ports: [{'model_name': 'Wafer', 'gpu': '0', 'port': 8015, 'container_id': '9cf068d1e536', 'pid': 1848066}, {'model_name': 'Wafer', 'gpu': '0', 'port': 8019, 'container_id': '20f9360fc68a', 'pid': 1844552}, {'model_name': 'SelfRegulationSCP1', 'gpu': '1', 'port': 8017, 'container_id': '731b3625375d', 'pid': 1814693}]


{8015, 8017, 8019}

In [295]:
client.predict(timeseries, instance_new, index_new)

(8017, {'predictions': [[0.991511047, 0.00848888606]]})

In [ ]:
unloaded_ports = client.unload_models()
print("Unloaded ports:", unloaded_ports)

In [272]:
client.terminate_all_containers()

{'message': 'Forcefully terminated 6 containers',
 'terminated_containers': [{'container_id': '2d7feccf00af',
   'gpu': '0',
   'model_name': 'ArticularyWordRecognition',
   'pid': 1803952,
   'port': 8019},
  {'container_id': '2d7feccf00af',
   'gpu': '0',
   'model_name': 'ArticularyWordRecognition',
   'pid': 1803952,
   'port': 8019},
  {'container_id': '4b11b5f1897e',
   'gpu': '0',
   'model_name': 'ArticularyWordRecognition',
   'pid': 1799945,
   'port': 8015},
  {'container_id': '4b11b5f1897e',
   'gpu': '0',
   'model_name': 'ArticularyWordRecognition',
   'pid': 1799945,
   'port': 8015},
  {'container_id': 'ec06e54f7771',
   'gpu': '0',
   'model_name': 'ArticularyWordRecognition',
   'pid': 1789013,
   'port': 8017},
  {'container_id': 'ec06e54f7771',
   'gpu': '0',
   'model_name': 'ArticularyWordRecognition',
   'pid': 1789013,
   'port': 8017}]}

In [167]:
client.used_ports

{8501, 8503}

##### Anchor code

In [17]:
def to_number(value):
    if isinstance(value, np.float64):
        return float(value)
    elif isinstance(value, np.ndarray) and value.size == 1:
        return float(value[0])
    elif isinstance(value, (int, float)):
        return value
    else:
        raise ValueError("Unsupported type: value must be a number or a single-element numpy array")


def convert_alibi_conditions_to_dict(conditions, feature_names):
    rule_dict = {}
    for feature_name in feature_names:
        for rule in conditions:
            if f"{feature_name} " in f"{rule} ":
                processed_rules = []
                for op in ["> ", ">= ", "< ", "<= "]:
                    if op in rule:
                        _rules = rule.split(op)
                        # print(_rules)
                        if len(_rules) == 2:
                            if feature_name in _rules[0]:
                                processed_rules.append((op + _rules[1]).replace(' ', ''))
                            elif feature_name in _rules[1]:
                                result = None
                                if op == "> ":
                                    result = "< "
                                elif op == ">= ":
                                    result = "<= "
                                elif op == "< ":
                                    result = "> "
                                elif op == "<= ":
                                    result = ">= "
                                processed_rules.append((result + _rules[0]).replace(' ', ''))
                        elif len(_rules) == 3:
                            if feature_name in _rules[1]:
                                processed_rules.append((op + _rules[2]).replace(' ', ''))
                                result = None
                                if op == "> ":
                                    result = "< "
                                elif op == ">= ":
                                    result = "<= "
                                elif op == "< ":
                                    result = "> "
                                elif op == "<= ":
                                    result = ">= "
                                processed_rules.append((result + _rules[0]).replace(' ', ''))

                rule_dict[feature_name] = processed_rules

    return rule_dict


def generate_anchor_explanation(
        model_name: str, directory: str, index, ts_df, train_data: np.ndarray, serv_port: int,
        serv_client: ModelServingClient, model_path: str, class_names: List[str], feature_names, n_timesteps,
        n_features,
        anchor_threshold: Optional[float], alt_method: bool, max_attempts=3, timeout_dur: int = 25, do_print=False,
        not_scaled_instance=None, prediction=None):
    """
    Generates an anchor explanation for a selected observation in a DataFrame.

    :param model_name: 
    :param directory: 
    :param index: Index of the observation to explain in the DataFrame.
    :param ts_df: DataFrame containing the observations.
    :param train_data: np.ndarray.
    :param serv_port: Port of the model server's prediction endpoint.
    :param serv_client: 
    :param model_path: 
    :param class_names: 
    :param feature_names: List of all feature names.
    :param n_timesteps: Number of timesteps in the input data.
    :param n_features: Number of features in the input data.
    :param alt_method: 
    :param max_attempts: Maximum number of attempts to generate a satisfactory explanation.
    :param timeout_dur: 
    :param do_print: If True, prints the generated explanation and associated metrics.
    :param not_scaled_instance: No need to pass ts_df. 
    :param prediction: Predicted label for the observation; if None, the model's prediction will be used.

    :return: Dictionary containing the generated explanation, including the rule, predicted class, and confidence score.
    """
    # print(f"Generating anchor explanation for {index}")
    explanation = {"index": index, "success": False}

    def server_predict(data: np.ndarray) -> np.ndarray:
        """
        Sends data to a TensorFlow model server and retrieves the prediction.
    
        :param data: Input data as a NumPy array, compatible with the model's expected shape.
        :return: Model predictions as a NumPy array, in the same format as TensorFlow's `model.predict`.
        """

        serv_port = -1
        repetition = 0
        while True:
            try:
                serv_port, response = serv_client.predict(model_name, data, index)
                if "predictions" in response:
                    # print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Prediction successful. Port - {serv_port}")
                    if repetition > 10:
                        print(
                            f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {repetition + 1} - Recovery SUCCESS, port - {serv_port}")
                    break
                else:
                    print(f"! {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Attempt {repetition + 1}: "
                          f"! Server returned status {response['error']}. Port - {serv_port}")
                    pass

            except Exception as e:
                print(f"! {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Attempt {repetition + 1}: "
                      f"Request exception - {e}. Port - {serv_port}")
                serv_port = -1

            if repetition % 20 == 0 and repetition > 0:
                print(
                    f"! {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {repetition + 1} - Will REBOOT SERVER {serv_port}")
                if serv_port > 0:
                    serv_client.terminate_container(serv_port)
                    time.sleep(5)
                else:
                    serv_port = serv_client.base_port

                serv_client.update_used_ports(model_name)

                serv_client.used_ports.discard(serv_port)
                serv_client.checked_ports.discard(serv_port)
                port = serv_client.find_available_port(serv_port)

                method = serv_client.method or "docker"
                grow_gpu_mem = serv_client.grow_gpu_mem or "true"
                gpu = serv_client.get_gpu_with_most_memory()
                print(
                    f"! {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {repetition + 1} - Kill server on port {serv_port} and create on {port}, gpu {gpu}, method {method}")

                external_model_path = os.path.join("/opt/jupyterhub/fast/shared_dir/UCI-Benchmark", 'ds', directory,
                                                   model_name,
                                                   "model_tf")

                serv_client.load_model(model_name, external_model_path, port, gpu, method, grow_gpu_mem)

            repetition += 1
            time.sleep(5)

        predictions = response.get("predictions", None)
        if predictions is None:
            info = f"! STOP {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - No predictions found in the server response. Port - {serv_port}"
            print(info)
            sys.exit(info)
            # raise ValueError("No predictions found in the server response.")

        predictions = np.array(predictions)

        if random.random() < .0001:
            print(str(datetime.now().strftime('%Y-%m-%d %H:%M:%S')), "-", serv_port, ": alive, observ. index:", index,
                  ", prediction shape:", predictions.shape, ", up to 3 first values:", predictions.flatten()[:3])
        return predictions

    predict_fn = lambda x: server_predict(x.reshape(-1, n_timesteps, n_features)).argmax(axis=1)
    if not alt_method:
        explainer = AnchorTabular(predict_fn, feature_names)
        explainer.fit(train_data)  # , disc_perc=(25, 50, 75)
    else:
        explainer = anchor_tabular.AnchorTabularExplainer(
            class_names=class_names,
            feature_names=feature_names,
            train_data=train_data,
            discretizer='quartile')
        print(str(datetime.now().strftime('%Y-%m-%d %H:%M:%S')), "-", serv_port, "-",
              "Using ALTERNATIVE explainer: observ. index:", index, ", explainer:", "marcotcr")
    try:
        instance_values = not_scaled_instance if not_scaled_instance is not None else ts_df.loc[
            index, feature_names].values

        if prediction is None:
            prediction = server_predict(instance_values.reshape(1, n_timesteps, n_features)).argmax(axis=1)[0]
        prediction = str(int(prediction))
        explanation["prediction"] = prediction
        if index < 3:
            print(f"Prediction for {index} is {prediction}")

        best_exp = None
        best_metric_for_exp = -1

        increment_step = (anchor_threshold + 0.5) / max_attempts
        for attempt in range(max_attempts):
            if anchor_threshold is not None:
                current_threshold = (attempt + 1) * increment_step
                threshold_value = min(current_threshold, anchor_threshold)
                if not alt_method:
                    exp = explainer.explain(instance_values, threshold=threshold_value)
                else:
                    exp = explainer.explain_instance(instance_values, predict_fn, threshold=threshold_value)
            else:
                if not alt_method:
                    exp = explainer.explain(instance_values)
                else:
                    exp = explainer.explain_instance(instance_values, predict_fn)
            if not alt_method:
                coverage, precision, exp_count = to_number(exp.coverage), to_number(exp.precision), len(exp.anchor)
            else:
                coverage, precision, exp_count = exp.coverage(), exp.precision(), len(exp.names())
            if coverage * precision * exp_count > best_metric_for_exp:
                best_exp = exp
                best_metric_for_exp = coverage * precision * exp_count

        if do_print:
            print("For observation #", index, "prediction =", f"Class {prediction}")
            print('Anchor: %s' % (' AND '.join(best_exp.anchor)))
            print('Precision: %.2f' % best_exp.precision)
            print('Coverage: %.2f' % best_exp.coverage)

        if best_exp is not None:
            if not alt_method:
                coverage, precision, exp_count, rules = to_number(best_exp.coverage), to_number(
                    best_exp.precision), len(best_exp.anchor), best_exp.anchor
            else:
                coverage, precision, exp_count, rules = best_exp.coverage(), best_exp.precision(), len(
                    best_exp.names()), best_exp.names()
            rule_dict = convert_alibi_conditions_to_dict(rules, feature_names)
            explanation['rule'] = rule_dict
            if not alt_method:
                explanation['anchor_method'] = "alibi"
            else:
                explanation['anchor_method'] = "marcotcr"
            explanation['anchor'] = rules
            explanation['confidence'] = precision
            explanation['coverage'] = coverage
            explanation['exp_count'] = exp_count
            explanation['success'] = True

    except Exception as e:
        print("!", str(datetime.now().strftime('%Y-%m-%d %H:%M:%S')), "-", serv_port, ": observ. index:", index,
              ", error:", str(e))
        explanation["error"] = str(e)

    return [explanation]  # For ANCHOR there is only one rule, but for DICE could be > 1 counterfactuals


def anchor_pool_worker(
        model_name: str, directory: str, available_ports: Optional[List[int]], msclient: ModelServingClient,
        port_queue: Optional[Queue], port_lock: Optional[Lock], index: int, instance_data: np.ndarray,
        feature_names: List[str], class_names: List[str], train_data: np.ndarray, n_features: int, n_timesteps: int,
        anchor_threshold: Optional[float], max_attempts: int, dataset_info: str, model_path: str, timeout_duration: int,
        alt_timeout_duration: int, ):
    """Wrapper for multiprocessing to generate explanations, with rotated REST calls on available ports."""

    explanation = [{"index": index, "success": False}]

    word_length = random.randint(3, 5)
    worker_name = ''.join(random.choices(string.ascii_uppercase, k=word_length))

    for alt_method, t_out in zip([False, False, True, True],
                                 [timeout_duration, 2 * timeout_duration, alt_timeout_duration,
                                  2 * alt_timeout_duration]):
        explanation[0]["success"] = False
        if not alt_method:
            explanation[0]['anchor_method'] = "alibi"
        else:
            print(
                f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - starting ALTERNATIVE method for obs. {index} of {dataset_info}.")
            explanation[0]['anchor_method'] = "marcotcr"

        with ProcessPoolExecutor(max_workers=1) as executor:
            future = executor.submit(generate_anchor_explanation,
                                     model_name=model_name,
                                     directory=directory,
                                     index=index,
                                     ts_df=None,
                                     train_data=train_data,
                                     serv_port=-1,
                                     serv_client=msclient,
                                     model_path=model_path,
                                     class_names=class_names,
                                     feature_names=feature_names,
                                     n_features=n_features,
                                     n_timesteps=n_timesteps,
                                     anchor_threshold=anchor_threshold,
                                     alt_method=alt_method,
                                     max_attempts=max_attempts,
                                     not_scaled_instance=instance_data)
            try:
                result = future.result(timeout=t_out)
                explanation = result
                if explanation[0].get("success", False):
                    if index < 3 or random.random() < .01:
                        print(
                            f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Explanation for sample {index} of {dataset_info} with alt_method={alt_method}: {explanation}")
                        explanation[0].pop("error", None)
                    break
            except FuturesTimeoutError:
                print(
                    f"! {worker_name} - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Timeout for sample {index} of {dataset_info} with alt_method={alt_method}.")
                explanation[0]["error"] = "FuturesTimeoutError"
            except Exception as e:
                error_trace = traceback.format_exc()
                print(
                    f"! {worker_name} - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: ERROR in anchor_pool_worker for obs. {index} of {dataset_info} with alt_method={alt_method}: {str(e)}, stack: {error_trace}.")
                explanation[0]["error"] = f"{str(e)},\n{error_trace}"

    if not explanation[0]["success"]:
        print(
            f"! {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Explanation for sample {index} of {dataset_info}: {explanation}")

    return explanation


def compute_anchor(
        port_queue: Optional[Queue],
        port_lock: Optional[Lock],
        model_name: str,
        directory: str,
        model_path: str,
        model: Any,
        available_ports: Optional[List[int]],
        trainX: np.ndarray,
        testX: np.ndarray,
        bg_data: np.ndarray,
        parallel: bool,
        timeout_duration: int,
        alt_timeout_duration: int,
        max_attempts: int = 3,
) -> Tuple[List[Dict[str, Any]], List[Dict[str, Any]]]:
    """
    Compute anchor explanations for train and test sets.

    :param port_queue:  
    :param port_lock:  
    :param model_name:  
    :param directory:  
    :param model_path:  
    :param model: To know the shape.
    :param available_ports: List of ports for REST API model servers.
    :param trainX: Training dataset (samples x timesteps x features).
    :param testX: Test dataset (samples x timesteps x features).
    :param bg_data: Background data for the explainer.
    :param max_attempts: Maximum number of attempts to generate a satisfactory explanation.

    :return: Tuple containing lists of anchor explanations for train and test sets.
    """
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    trainX_reshaped = trainX.reshape(-1, n_timesteps * n_features)
    testX_reshaped = testX.reshape(-1, n_timesteps * n_features)

    # Define feature names and class names
    feature_names = [f'feature_{i}' for i in range(n_timesteps * n_features)]
    class_names = [f'class_{i}' for i in range(model.output_shape[-1])]
    train_data = bg_data.reshape(-1, n_timesteps * n_features)

    # Generate anchor explanations for train and test sets
    anchor_explanations_train, anchor_explanations_test = [], []
    if parallel:
        _client = ModelServingClient(MODEL_MANAGER, base_port=8015)
        _client.update_used_ports(model_name)
        available_ports = list(_client.used_ports)

        # Determine the number of cores
        # total_cores = multiprocessing.cpu_count()
        # num_cores = int(5 / 11 * total_cores)
        num_cores = int(11 / 11 * len(available_ports) - 1)  #  5 / 11 * ... - 2
        # num_cores = 48
        print(f"Using {num_cores} cores for parallel processing")

        # Parallel computation of anchor explanations
        for dataset, dataX_reshaped, anchor_explanations in zip(
                ["Train", "Test"], [trainX_reshaped, testX_reshaped],
                [anchor_explanations_train, anchor_explanations_test]):
            print(f"{dataset} set (parallel)")
            with ProcessPoolExecutor(max_workers=num_cores) as executor:
                futures = []
                for i in range(len(dataX_reshaped)):
                    future = executor.submit(anchor_pool_worker,
                                             model_name, directory, available_ports, _client, port_queue, port_lock, i,
                                             dataX_reshaped[i].reshape(1, -1), feature_names, class_names, train_data,
                                             n_features, n_timesteps, 0.95, max_attempts, dataset, model_path,
                                             timeout_duration, alt_timeout_duration)
                    futures.append((i, future))

                final_results = []
                for index, future in futures:
                    # index = -1
                    try:
                        explanation = future.result(timeout=(4 * timeout_duration + 4 * alt_timeout_duration + 30))
                    except multiprocessing.context.TimeoutError:
                        print(
                            f"! {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: TIMEOUT in multiprocessing.Pool for obs. {index}.")
                        explanation = [{"index": index, "success": False, "error": "TIMEOUT in multiprocessing.Pool"}]

                    except Exception as e:
                        error_trace = traceback.format_exc()
                        print(
                            f"! {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}: ERROR in multiprocessing.Pool for obs. {index}: {str(e)}, stack: {error_trace}.")
                        explanation = [{"index": index, "success": False, "error": f"{str(e)},\n{error_trace}"}]

                    if index % 25 == 0 and index > 0:
                        print(f"Saving {dataset}, obs= {index}, explanation= {explanation}")
                        cache = [exp for i, exp in final_results if exp[0]["success"]]
                        if dataset == "Train":
                            save_bundle(None, None, None, None, None, cache, None, model_name,
                                        dir=os.path.join(data_path, 'ds', directory), explain_prefix="CACHE_av",
                                        only_explain=True)
                        else:
                            save_bundle(None, None, None, None, None, None, cache, model_name,
                                        dir=os.path.join(data_path, 'ds', directory), explain_prefix="CACHE_av",
                                        only_explain=True)

                    final_results.append((index, explanation))

            # Sort results by index to ensure the original order is maintained
            print(f"Sorting {dataset}, #", len(final_results))
            final_results.sort(key=lambda x: x[0])
            # Collect results 
            for i, explanation in final_results:
                anchor_explanations.append(explanation)
            print(
                f"--------------------------------------- {directory} - {model_name} - {dataset}: --------------------------------------- ")
            summarize_anchor_explanations(anchor_explanations)

        print("DONE")

    else:
        raise NotImplementedError("Serial")

    return anchor_explanations_train, anchor_explanations_test


def summarize_anchor_explanations(anchor_explanations: List[Dict[str, any]]) -> None:
    """
    Summarizes success and failure rates of anchor explanations and displays 
    a breakdown of failure types with percentages.

    :param anchor_explanations: List of anchor explanations with each explanation 
                                containing a "success" key and optionally an "error" message.
    """
    total_explanations = len(anchor_explanations)
    if total_explanations == 0:
        print("No anchor explanations to summarize.")
        return

    # Count successes and failures
    success_count = sum(1 for explanations in anchor_explanations if explanations[0].get("success"))
    failure_count = total_explanations - success_count

    # Group errors by type
    error_types = {}
    for explanations in anchor_explanations:
        explanation = explanations[0]
        if not explanation.get("success"):
            error_message = explanation.get("error", "Unknown error")
            if error_message not in error_types:
                error_types[error_message] = 0
            error_types[error_message] += 1

    # Calculate percentages
    success_percentage = (success_count / total_explanations) * 100
    failure_percentage = (failure_count / total_explanations) * 100

    # Display results
    print(f"\nSummary of Anchor Explanations (Total: {total_explanations}):")
    print(f"  - Success: {success_count} ({success_percentage:.2f}%)")
    print(f"  - Failure: {failure_count} ({failure_percentage:.2f}%)")

    # Display failure types with percentages
    print("\nFailure Types:")
    for error_type, count in error_types.items():
        error_percentage = (count / total_explanations) * 100
        print(f"  - {error_type}: {count} ({error_percentage:.2f}%)")

#### Setup and test

In [18]:
bg_size = 1000
window_len = 23
stride = 10
explanation_method = 'NA'  # or 'window' if needed for SHAP window-based explanations
absshap = True
shap_enabled = False  # SHAP disabled
lime_enabled = False  # LIME enabled
anchor_enabled = True  # ANCHOR enabled
explain_prefix = "av"

print("path:", data_path, ", batch:", batch_size, ", background:", bg_size, ", window:", window_len, ", stride:",
      stride, ", method:", explanation_method, ", absshap:", absshap, ", SHAP:", shap_enabled, ", LIME:", lime_enabled,
      ", ANCHOR:", anchor_enabled, ", prefix:", explain_prefix)

summary = []

path: shared/UCI-Benchmark/ , batch: 64 , background: 1000 , window: 23 , stride: 10 , method: NA , absshap: True , SHAP: False , LIME: False , ANCHOR: True , prefix: av


In [19]:
datasets_uni_minus_ChlorineConcentration_last_TwoPatterns_UWaveGestureLibraryXXX = [a for a in datasets_uni if
                                                                                    a not in [
                                                                                        "ChlorineConcentration",
                                                                                        "TwoPatterns",
                                                                                        "UWaveGestureLibraryAll",
                                                                                        "UWaveGestureLibraryX",
                                                                                        "UWaveGestureLibraryY",
                                                                                        "UWaveGestureLibraryZ",
                                                                                    ]] + [
                                                                                       "UWaveGestureLibraryX",
                                                                                       "UWaveGestureLibraryY",
                                                                                       "UWaveGestureLibraryZ",
                                                                                       "UWaveGestureLibraryAll",
                                                                                       "TwoPatterns",
                                                                                   ]

print(datasets_uni_minus_ChlorineConcentration_last_TwoPatterns_UWaveGestureLibraryXXX)

['DodgerLoopGame', 'ProximalPhalanxTW', 'DodgerLoopDay', 'ECGFiveDays', 'UMD', 'Plane', 'ECG200', 'GunPointAgeSpan', 'DiatomSizeReduction', 'MedicalImages', 'Meat', 'Trace', 'MiddlePhalanxOutlineAgeGroup', 'Chinatown', 'DistalPhalanxOutlineAgeGroup', 'WormsTwoClass', 'DistalPhalanxOutlineCorrect', 'Strawberry', 'OliveOil', 'SmoothSubspace', 'Fungi', 'ElectricDevices', 'SwedishLeaf', 'CricketX', 'ECG5000', 'PhalangesOutlinesCorrect', 'FaceFour', 'SyntheticControl', 'FordA', 'PowerCons', 'BeetleFly', 'GunPointMaleVersusFemale', 'Yoga', 'Herring', 'Crop', 'RefrigerationDevices', 'Worms', 'OSULeaf', 'ItalyPowerDemand', 'GunPoint', 'CBF', 'Symbols', 'ToeSegmentation2', 'TwoLeadECG', 'SmallKitchenAppliances', 'ShapesAll', 'ScreenType', 'Computers', 'CricketY', 'SonyAIBORobotSurface2', 'Wafer', 'MiddlePhalanxOutlineCorrect', 'Lightning2', 'Wine', 'ProximalPhalanxOutlineCorrect', 'ShapeletSim', 'CricketZ', 'BME', 'Beef', 'DodgerLoopWeekend', 'ProximalPhalanxOutlineAgeGroup', 'LargeKitchenAppli

In [44]:
# p = 'Wafer'
# directory = 'univariate'
p = "Libras"  # 'SelfRegulationSCP1'
directory = 'multivariate'

model_temp, _, trainX, trainy, testX, testy, _, _ = load_bundle(p, dir=os.path.join(data_path, 'ds', directory),
                                                                load_model=True)
model_path = os.path.join(data_path, 'ds', directory, p)
print(trainX.shape, trainX[:10].shape, model_path)

Loading model in tf format...


2024-11-30 20:44:28.198029: W tensorflow/c/c_api.cc:304] Operation '{name:'AssignVariableOp_330' id:8499 op device:{requested: '/device:CPU:0', assigned: ''} def:{{{node AssignVariableOp_330}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false, _device="/device:CPU:0"](bias_33, Identity_330)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


(270, 45, 2) (10, 45, 2) shared/UCI-Benchmark/ds/multivariate/Libras


In [45]:
ms_client = ModelServingClient(MODEL_MANAGER, base_port=8015)
ms_client.update_used_ports(p)

Updated used ports: []


In [43]:
# ms_client.terminate_all_containers(p)

{'message': 'Forcefully terminated 18 containers',
 'terminated_containers': [{'container_id': 'ca83faa5d4c2',
   'gpu': '0',
   'model_name': 'SelfRegulationSCP1',
   'pid': 3252436,
   'port': 8055},
  {'container_id': 'e4cee15b84ef',
   'gpu': '1',
   'model_name': 'SelfRegulationSCP1',
   'pid': 3249115,
   'port': 8053},
  {'container_id': '5ee470892995',
   'gpu': '0',
   'model_name': 'SelfRegulationSCP1',
   'pid': 3245749,
   'port': 8051},
  {'container_id': 'a2b7b4b043ca',
   'gpu': '1',
   'model_name': 'SelfRegulationSCP1',
   'pid': 3242367,
   'port': 8049},
  {'container_id': 'eb715190ed2a',
   'gpu': '0',
   'model_name': 'SelfRegulationSCP1',
   'pid': 3238995,
   'port': 8047},
  {'container_id': 'd01c53a7ed52',
   'gpu': '1',
   'model_name': 'SelfRegulationSCP1',
   'pid': 3235660,
   'port': 8045},
  {'container_id': 'b2814806b585',
   'gpu': '0',
   'model_name': 'SelfRegulationSCP1',
   'pid': 3232285,
   'port': 8043},
  {'container_id': 'cbaab875a5d5',
   'gpu

In [49]:
last_instance = trainX[-1]
last_instance_for_predict = np.expand_dims(last_instance, axis=0)
model_info, tf_model_server_ports = ms_client.load_models(
    directory, p, data=last_instance_for_predict, count=9 * 2, grow_gpu_mem="true", method="docker", gpu=None,
    num_samples=1_000, req_free_memory_gb=1)

memory_info: {'0': '15512 MiB', '1': '15638 MiB'}
method: docker , port: 8091 , gpu: 1 , model_name: Libras , test_data: (1001, 45, 2)
memory_info: {'0': '15512 MiB', '1': '15284 MiB'}
method: docker , port: 8093 , gpu: 0 , model_name: Libras , test_data: (1001, 45, 2)
memory_info: {'0': '15095 MiB', '1': '15284 MiB'}
method: docker , port: 8095 , gpu: 1 , model_name: Libras , test_data: (1001, 45, 2)
memory_info: {'0': '15095 MiB', '1': '14931 MiB'}
method: docker , port: 8097 , gpu: 0 , model_name: Libras , test_data: (1001, 45, 2)
memory_info: {'0': '14678 MiB', '1': '14931 MiB'}
method: docker , port: 8099 , gpu: 1 , model_name: Libras , test_data: (1001, 45, 2)
memory_info: {'0': '14678 MiB', '1': '14578 MiB'}
method: docker , port: 8101 , gpu: 0 , model_name: Libras , test_data: (1001, 45, 2)
memory_info: {'0': '14261 MiB', '1': '14578 MiB'}
method: docker , port: 8103 , gpu: 1 , model_name: Libras , test_data: (1001, 45, 2)
memory_info: {'0': '14261 MiB', '1': '14225 MiB'}
metho

In [47]:
available_ports = list(ms_client.used_ports)
if len(available_ports) == 0:
    raise Exception("No available ports")
print("We have ", len(available_ports), "available ports")

We have  18 available ports


In [40]:
ms_client.terminate_all_containers("test")

{'message': 'Forcefully terminated 0 containers', 'terminated_containers': []}

In [ ]:
test_N = 10
sv_tr, sv_ts, lime_tr, lime_ts, anchor_tr, anchor_ts = process_dataset(
    None, None, p, directory, model_path, model_temp, available_ports, trainX[:test_N], trainy[:test_N], testX[:test_N],
    testy[:test_N], bg_size, window_len, stride, explanation_method, absshap, shap_enabled, lime_enabled,
    anchor_enabled, timeout_duration=60, alt_timeout_duration=60, parallel=True)

print(f'Saving bundle... {p},  SHAP: {shap_enabled}, LIME: {lime_enabled}, ANCHOR: {anchor_enabled}')
save_bundle(None, trainX, trainy, testX, testy, anchor_tr, anchor_ts, p,
            dir=os.path.join(data_path, 'ds', directory), explain_prefix="TEST_" + explain_prefix, only_explain=True)

In [54]:
anchor_ts

[[{'index': -2, 'success': False, 'error': 'TIMEOUT in multiprocessing.Pool'}],
 [{'index': -8, 'success': False, 'error': 'TIMEOUT in multiprocessing.Pool'}],
 [{'index': 0,
   'success': True,
   'prediction': '1',
   'rule': {},
   'anchor_method': 'alibi',
   'anchor': [],
   'confidence': 1.0,
   'coverage': 1,
   'exp_count': 0}],
 [{'index': 1,
   'success': True,
   'prediction': '1',
   'rule': {},
   'anchor_method': 'alibi',
   'anchor': [],
   'confidence': 1.0,
   'coverage': 1,
   'exp_count': 0}],
 [{'index': 3,
   'success': True,
   'prediction': '1',
   'rule': {},
   'anchor_method': 'alibi',
   'anchor': [],
   'confidence': 1.0,
   'coverage': 1,
   'exp_count': 0}],
 [{'index': 4,
   'success': True,
   'prediction': '1',
   'rule': {},
   'anchor_method': 'alibi',
   'anchor': [],
   'confidence': 1.0,
   'coverage': 1,
   'exp_count': 0}],
 [{'index': 5,
   'success': True,
   'prediction': '1',
   'rule': {},
   'anchor_method': 'alibi',
   'anchor': [],
   'co

In [73]:
p not in ['Wafer', 'DodgerLoopGame']

False

In [19]:
ms_client = ModelServingClient(MODEL_MANAGER, base_port=8015)
if p not in ['Wafer', 'DodgerLoopGame']:
    ms_client.terminate_all_containers(p)
ms_client.update_used_ports()
print("We have ", len(list(ms_client.used_ports)), "available ports")

Updated used ports: []
We have  0 available ports


In [83]:
ms_client.get_gpu_with_most_memory()
print(ms_client.has_sufficient_gpu_memory("0", 1))
print(ms_client.has_sufficient_gpu_memory("0", 2))
print(ms_client.has_sufficient_gpu_memory("1", 1))


memory_info: {'0': '1960 MiB', '1': '631 MiB'}
True
False
False


In [84]:
# ms_client.load_models(directory, p, data=last_instance_for_predict, count=1, grow_gpu_mem="true", method="docker",
#                       gpu=None, num_samples=3_000)

memory_info: {'0': '1960 MiB', '1': '631 MiB'}
method: docker , port: 8015 , gpu: 0 , model_name: DodgerLoopGame , test_data: (3001, 288, 1)


([{'model_name': 'DodgerLoopGame',
   'port': 8015,
   'gpu': '0',
   'load_time': 0.9104182720184326,
   'status': 'loaded'}],
 [8119,
  8121,
  8123,
  8125,
  8127,
  8129,
  8131,
  8133,
  8135,
  8137,
  8139,
  8141,
  8143,
  8015,
  8145,
  8147,
  8149,
  8151,
  8153,
  8155,
  8157,
  8159,
  8161,
  8163,
  8165,
  8167,
  8169,
  8171,
  8173,
  8175,
  8177,
  8181,
  8183,
  8185,
  8187,
  8189])

#### Univariate

In [45]:
# ms_client = ModelServingClient(MODEL_MANAGER, base_port=9015)
ms_client.terminate_all_containers()

{'message': 'Forcefully terminated 17 containers',
 'terminated_containers': [{'container_id': '6dbc3f9357ef',
   'gpu': '0',
   'model_name': 'SelfRegulationSCP2',
   'pid': 1925257,
   'port': 10023},
  {'container_id': '373e2c2dcf97',
   'gpu': '0',
   'model_name': 'Meat',
   'pid': 1921881,
   'port': 9037},
  {'container_id': '901a5e593005',
   'gpu': '1',
   'model_name': 'SelfRegulationSCP2',
   'pid': 1918704,
   'port': 10021},
  {'container_id': '3bdfa68de982',
   'gpu': '1',
   'model_name': 'Meat',
   'pid': 1915404,
   'port': 9035},
  {'container_id': 'ff6150c5fc64',
   'gpu': '0',
   'model_name': 'SelfRegulationSCP2',
   'pid': 1911825,
   'port': 10019},
  {'container_id': '71394f72ea13',
   'gpu': '0',
   'model_name': 'Meat',
   'pid': 1908645,
   'port': 9033},
  {'container_id': '2bede1b98bd0',
   'gpu': '1',
   'model_name': 'SelfRegulationSCP2',
   'pid': 1903457,
   'port': 10017},
  {'container_id': '4c3b3a3815f5',
   'gpu': '1',
   'model_name': 'Meat',
   'p

In [ ]:
N = 10 * 2  # Wafer: 26, DodgerLoopGame: 18
directory = 'univariate'
ms_client = ModelServingClient(MODEL_MANAGER, base_port=9015)
# ms_client.terminate_all_containers()

for p in ['Wafer'] + datasets_uni_minus_ChlorineConcentration_last_TwoPatterns_UWaveGestureLibraryXXX:
    print(f'Loading... {p}')

    explain_tr_path = os.path.join(data_path, 'ds', directory, p, f'{explain_prefix}tr.pickle')
    explain_ts_path = os.path.join(data_path, 'ds', directory, p, f'{explain_prefix}ts.pickle')
    if os.path.exists(explain_tr_path) and os.path.exists(explain_ts_path):
        if os.path.getsize(explain_tr_path) > 1024 and os.path.getsize(explain_ts_path) > 1024:
            print(
                f"Omitting {p}, explainers {directory} SHAP:{shap_enabled} LIME:{lime_enabled} ANCHOR:{anchor_enabled} already calculated.")
            continue

    model_path = os.path.join(data_path, 'ds', directory, p)
    if p not in ['Wafer']:
        model_temp, _, trainX, trainy, testX, testy, _, _ = load_bundle(p, dir=os.path.join(data_path, 'ds', directory),
                                                                        load_model=True)
    last_instance = trainX[-1]
    last_instance_for_predict = np.expand_dims(last_instance, axis=0)

    if p not in ['Wafer', 'DodgerLoopGame']:
        model_info, tf_model_server_ports = ms_client.load_models(
            directory, p, data=last_instance_for_predict, count=N, grow_gpu_mem="true", method="docker", gpu=None,
            num_samples=100, req_free_memory_gb=1)

    ms_client.update_used_ports(p)
    available_ports = list(ms_client.used_ports)
    if len(available_ports) == 0:
        raise Exception("No available ports")
    print("We have ", len(available_ports), "available ports")

    sv_tr, sv_ts, lime_tr, lime_ts, anchor_tr, anchor_ts = process_dataset(
        None, None, p, directory, model_path, model_temp, available_ports, trainX, trainy, testX, testy,
        bg_size, window_len, stride, explanation_method, absshap, shap_enabled, lime_enabled, anchor_enabled,
        timeout_duration=int(2 * 60.0 * 60), alt_timeout_duration=int(2 * 60.0 * 60), parallel=True)

    # _, score = evaluate_model(model_temp, testX, testy, batch_size, None)
    # score = score * 100.0
    # print('>Accuracy: %.3f' % (score))

    print(f'Saving bundle... {p},  SHAP: {shap_enabled}, LIME: {lime_enabled}, ANCHOR: {anchor_enabled}')
    save_bundle(None, trainX, trainy, testX, testy, anchor_tr, anchor_ts, p,
                dir=os.path.join(data_path, 'ds', directory), explain_prefix=explain_prefix, only_explain=True)
    # summary.append([directory, p, score, 'OK'])

    tf.keras.backend.clear_session()
    del model_temp, trainX, trainy, testX, testy, sv_tr, sv_ts, lime_tr, lime_ts
    gc.collect()
    ms_client.terminate_all_containers(p)


# Explanation for train sample 0: [{'index': 0, 'success': True, 'prediction': '0', 'rule': {'feature_2': ['<=0.95'], 'feature_20': ['<=0.95'], 'feature_26': ['<=-0.50'], 'feature_200': ['<=0.95'], 'feature_264': ['<=-0.12'], 'feature_265': ['<=-0.50']}, 'confidence': 1.0}]
# Explanation for train sample 1: [{'index': 1, 'success': True, 'prediction': '1', 'rule': {'feature_2': ['>0.57'], 'feature_20': ['>1.34'], 'feature_26': ['>0.57'], 'feature_201': ['>1.34'], 'feature_261': ['>0.57']}, 'confidence': 0.9803157093952073}]
# Explanation for train sample 2: [{'index': 2, 'success': True, 'prediction': '0', 'rule': {'feature_2': ['>0.72'], 'feature_22': ['>0.72'], 'feature_26': ['<=-0.50'], 'feature_224': ['>0.72'], 'feature_264': ['<=-0.12'], 'feature_265': ['<=-0.50']}, 'confidence': 1.0}]    

Loading... Wafer
Omitting Wafer, explainers univariate SHAP:False LIME:False ANCHOR:True already calculated.
Loading... DodgerLoopGame
Omitting DodgerLoopGame, explainers univariate SHAP:False LIME:False ANCHOR:True already calculated.
Loading... ProximalPhalanxTW
Omitting ProximalPhalanxTW, explainers univariate SHAP:False LIME:False ANCHOR:True already calculated.
Loading... DodgerLoopDay
Omitting DodgerLoopDay, explainers univariate SHAP:False LIME:False ANCHOR:True already calculated.
Loading... ECGFiveDays
Omitting ECGFiveDays, explainers univariate SHAP:False LIME:False ANCHOR:True already calculated.
Loading... UMD
Omitting UMD, explainers univariate SHAP:False LIME:False ANCHOR:True already calculated.
Loading... Plane
Omitting Plane, explainers univariate SHAP:False LIME:False ANCHOR:True already calculated.
Loading... ECG200
Omitting ECG200, explainers univariate SHAP:False LIME:False ANCHOR:True already calculated.
Loading... GunPointAgeSpan
Omitting GunPointAgeSpan, explain

2025-03-18 11:16:27.783941: W tensorflow/c/c_api.cc:304] Operation '{name:'AssignVariableOp_103' id:2364 op device:{requested: '/device:CPU:0', assigned: ''} def:{{{node AssignVariableOp_103}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false, _device="/device:CPU:0"](conv_lstm1d/bias/v_2, Identity_101)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


memory_info: {'0': '12002 MiB', '1': '12002 MiB'}
method: docker , port: 9015 , gpu: 0 , model_name: ElectricDevices , test_data: (101, 96, 1)
memory_info: {'0': '11393 MiB', '1': '12002 MiB'}
method: docker , port: 9017 , gpu: 1 , model_name: ElectricDevices , test_data: (101, 96, 1)
memory_info: {'0': '11393 MiB', '1': '11393 MiB'}
method: docker , port: 9019 , gpu: 0 , model_name: ElectricDevices , test_data: (101, 96, 1)
memory_info: {'0': '10784 MiB', '1': '11393 MiB'}
method: docker , port: 9021 , gpu: 1 , model_name: ElectricDevices , test_data: (101, 96, 1)
memory_info: {'0': '10784 MiB', '1': '10784 MiB'}
method: docker , port: 9023 , gpu: 0 , model_name: ElectricDevices , test_data: (101, 96, 1)
memory_info: {'0': '10174 MiB', '1': '10784 MiB'}
method: docker , port: 9025 , gpu: 1 , model_name: ElectricDevices , test_data: (101, 96, 1)
memory_info: {'0': '10174 MiB', '1': '10174 MiB'}
method: docker , port: 9027 , gpu: 0 , model_name: ElectricDevices , test_data: (101, 96, 1)

In [1]:
1

1

In [ ]:
# Wafer
# 2024-11-10 23:33:20 - Explanation for sample 0 of Train: [{'index': 0, 'success': True, 'prediction': '1', 'rule': {'feature_1': ['>-0.74'], 'feature_4': ['<=0.73'], 'feature_11': ['>-0.74'], 'feature_42': ['<=0.73'], 'feature_110': ['>-0.74']}, 'confidence': 0.9565217391304348, 'coverage': 0.7197}]
# 2024-11-12 11:12:59 - Explanation for sample 13 of Train: [{'index': 13, 'success': True, 'prediction': '1', 'rule': {'feature_1': ['>0.40'], 'feature_10': ['>0.40'], 'feature_106': ['>0.40']}, 'confidence': 0.9507829977628636, 'coverage': 0.5061}]


#### Multivariate

In [20]:
datasets_multi_not_list = [
    "FaceDetection",
    "EthanolConcentration",
    "HandMovementDirection",
    "SelfRegulationSCP1",
    "SelfRegulationSCP2",
]
datasets_multi_minus_FaceDetection_EthanolConcentration_last_HandMovementDirection = [a for a in datasets_multi if
                                                                                      a not in datasets_multi_not_list] + [
                                                                                         "HandMovementDirection",
                                                                                         "SelfRegulationSCP1",
                                                                                         "SelfRegulationSCP2"]
print(datasets_multi_minus_FaceDetection_EthanolConcentration_last_HandMovementDirection)

['ArticularyWordRecognition', 'Handwriting', 'BasicMotions', 'Heartbeat', 'SelfRegulationSCP2', 'NATOPS', 'ERing', 'FingerMovements', 'Epilepsy', 'Libras', 'AtrialFibrillation', 'PenDigits', 'Cricket', 'LSST', 'UWaveGestureLibrary', 'RacketSports', 'SelfRegulationSCP1', 'HandMovementDirection']


In [65]:
p

'Libras'

In [ ]:
ms_client = ModelServingClient(MODEL_MANAGER, base_port=12015)
# if p != 'SelfRegulationSCP1':
#     ms_client.terminate_all_containers(p)

N = 12 * 2
directory = 'multivariate'

# for p in datasets_uni_minus_ChlorineConcentration:
for p in ["Libras"] + datasets_multi_minus_FaceDetection_EthanolConcentration_last_HandMovementDirection:
    print(f'Loading... {p}')

    explain_tr_path = os.path.join(data_path, 'ds', directory, p, f'{explain_prefix}tr.pickle')
    explain_ts_path = os.path.join(data_path, 'ds', directory, p, f'{explain_prefix}ts.pickle')
    if os.path.exists(explain_tr_path) and os.path.exists(explain_ts_path):
        if os.path.getsize(explain_tr_path) > 1024 and os.path.getsize(explain_ts_path) > 1024:
            print(
                f"Omitting {p}, explainers {directory} SHAP:{shap_enabled} LIME:{lime_enabled} ANCHOR:{anchor_enabled} already calculated.")
            continue

    model_path = os.path.join(data_path, 'ds', directory, p)

    model_temp, _, trainX, trainy, testX, testy, _, _ = load_bundle(p, dir=os.path.join(data_path, 'ds', directory),
                                                                    load_model=True)
    last_instance = trainX[-1]
    last_instance_for_predict = np.expand_dims(last_instance, axis=0)

    if p != 'Libras':
        model_info, tf_model_server_ports = ms_client.load_models(
            directory, p, data=last_instance_for_predict, count=N, grow_gpu_mem="true", method="docker", gpu=None,
            num_samples=1_000, req_free_memory_gb=1)

    ms_client.update_used_ports(p)
    available_ports = list(ms_client.used_ports)
    if len(available_ports) == 0:
        raise Exception("No available ports")
    print("We have ", len(available_ports), "available ports:", available_ports)

    sv_tr, sv_ts, lime_tr, lime_ts, anchor_tr, anchor_ts = process_dataset(
        None, None, p, directory, model_path, model_temp, available_ports, trainX, trainy, testX, testy,
        bg_size, window_len, stride, explanation_method, absshap, shap_enabled, lime_enabled, anchor_enabled,
        timeout_duration=int(2 * 60.0 * 60), alt_timeout_duration=int(2 * 60.0 * 60), parallel=True)

    _, score = evaluate_model(model_temp, testX, testy, batch_size, None)
    score = score * 100.0
    print('>Accuracy: %.3f' % (score))

    print(f'Saving bundle... {p},  SHAP: {shap_enabled}, LIME: {lime_enabled}, ANCHOR: {anchor_enabled}')
    save_bundle(None, trainX, trainy, testX, testy, anchor_tr, anchor_ts, p,
                dir=os.path.join(data_path, 'ds', directory), explain_prefix=explain_prefix, only_explain=True)
    summary.append([directory, p, score, 'OK'])

    tf.keras.backend.clear_session()
    del model_temp, trainX, trainy, testX, testy, sv_tr, sv_ts, lime_tr, lime_ts
    gc.collect()
    ms_client.terminate_all_containers(p)

In [ ]:
pd.DataFrame(summary)

In [ ]:
pd.DataFrame(summary, columns=['dataset', 'score', 'status']).to_csv(data_path + '/ds/summary-multi-1.csv',
                                                                     index=False)